# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [ ]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

In [ ]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [ ]:
#TODO: Fetch and upload the data to AWS S3
from sagemaker.tuner import (
    CategoricalParameter,
    ContinousParameter,
    HyperparameterTunner
)

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/Pneumonia-data"
role = sagemaker.get_execution_role()


inputs = sagemaker_session.upload_data(path = '/Data', bucket = bucket, key_prefix = prefix)
print(inputs)

In [ ]:
import os 
os.environ["DEFAULT_S3_BUCKET"] = bucket

!aws s3 sync ./Data s3://${DEFAULT_S3_BUCKET}/dataset/

In [ ]:
import logging
from botocore.exceptions import  ClientError
import os

def upload_file(file_name, bucket, object_name = None):
    #if s3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)
    
    #upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [ ]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "lr": ContinousParameter(0.001, 0.1),
    "batch-size": CategoricalParameter([32, 64, 128, 256, 512])
}
objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitons = [{"Name": "average test loss", "Regex": "Test set: Average loss: ([0-9//.]+)"}]


In [ ]:
#TODO: Create estimators for your HPs
from sagemaker.pytorch import Pytorch

estimator = Pytorch(
    entry_point = "hpo.py",
    role = role,
    py_version = 'py36',
    framework_version = "1.8",
    instance_count = 1,
    instance_type = "ml.m5.large"
)# TODO: Your estimator here
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitons,
    max_jobs = 4,
    max_parallel_jobs = 2,
    objective_type = objective_type
)# TODO: Your HP tuner here

In [ ]:
# TODO: Fit your HP Tuner
tuner.fit({"training_data": inputs}) # TODO: Remember to include your data channels

In [ ]:
# TODO: Get the best estimators and the best HPs

best_estimator = #TODO

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks
from sagemaker.debugger import (
    Rule,
    DebuggerHookConfig, 
    rule_configs
)
from sagemaker.debugger import DebuggerHookConfig, ProfileConfig, FrameWorkProfile 

rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization())
]


hook_config = DebuggerHookConfig(
    hook_parameters = {"train.save_interval": "100", "eval.save_interval": "10"}
)

profiler_config = ProfileConfig(
    system_monitor_interval_millis = 500, framework_profile_params = FrameWorkProfile (num_steps = 10)
)

In [ ]:
# TODO: Create and fit an estimator
hyperparameters = {"model": "densenet121"}
estimator = {
    entry_point = "train_model.py",
    base_job_name = "train-model-debug",
    role = role,
    instance_count = 1,
    instance_type = "ml.m5.large",
    hyperparameters = hyperparameters,
    framework_version = "1.8",
    py_version = "py36",
    rules = rules,
    debugger_hook_config = hook_config
}# TODO: Your estimator here
estimator.fit(wait = True)

In [ ]:
# TODO: Plot a debugging output.

job_name = estimator.latest_training_job.name
client = estimator.sagemaker_session.sagemaker_client
description = client.describe_training_job(TrainingJobName = estimator.latest_training_job.name)

region = sagemaker_session.region_name

In [ ]:
from smdebug.trials import create_trial
from smdebug.core.modes import Modekeys

trial = create_trial(estimator.latest_job_debugger_artifacts_path())
trial.tensor_names()

print(len(trial.tensor("nll_loss_output_0").steps(mode = Modekeys.TRAIN)))
print(len(trial.tensor("nll_loss_output_0").steps(mode = Modekeys.EVAL)))

In [ ]:
def get_data(trial, tname, mode):
    tensor = trial.tensor(tname)
    steps = tensor.steps(mode = mode)
    vals = []
    for s in steps:
        vals.append(tensor.values(s, mode = mode))
    return steps, vals

In [ ]:
import matplotlib.pyplot as plt 
from mpl_tooltkits.axes_grid1 import host_subplot

def plot_tensor(trial, tensor_name):
    steps_train, vals_train = get_data(trial, tensor_name, model=Modekeys.TRAIN)
    print("loaded TRAIN data")
    steps_eval, vals_eval = get_data(trial, tensor_name, model=Modekeys.TRAIN)
    print("loaded EVAL data")

    fig = plt.figure(figsize = (10, 7))
    host = host_subplot(111)

    par = host.twiny()

    host.set_xlabel("Steps (TRAIN)")
    par.set_xlabel("Stpes (EVAL)")
    host.set_ylabel(tensor_name)

    (p1, ) = host.plot(steps_train, vals_train, label = tensor_name)
    print("Completed TRAIN plot")
    (p2, ) = par.plot(steps_eval, vals_eval, label = "Val_" + tensor_name)
    print("Completed EVAL plot")
    leg = plt.legend()

    host.xaxis.get_label().set_color(p1.get_color())
    leg.texts[0].set_color(p1.get_color())

    par.xaxis.get_label().set_color(p2.get_color())
    leg.texts[1].ser_color(p2.get_color())

    plt.ylabel(tensor_name)

    plt.show()


In [ ]:
plot_tensor(trial, "nn_loss_output_0")

In [ ]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob 

tj = TrainingJob(job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

In [ ]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader = None,
    select_dimensions = ["CPU", "GPU"],
    select_events = ["total"]
)

In [ ]:
rule_output_path = estimator.out_path + estimator.latest_training_job.job_name + "/rule-output"
print("Profile Report {}".format(rule_output_path))

In [ ]:
! aws s3 ls {rule_output_path} --recursive 

In [ ]:
! aws s3 cp {rule_output_path} ./ --recursive 

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output
import os

profiler_report_name = [ 
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

import IPython

IPython.display.HTML(filename = profiler_report_name + "/profile-output/profiler-report.html")

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy(
    initial_count = 1,
    instance_type = "ml.t2.meduim"
) # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()